In [24]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.metrics import confusion_matrix ,classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
def relu(x):
    assert isinstance(x, np.ndarray), "Input to ReLU must be a numpy array"
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU output must be non-negative"
    return result

def relu_derivative(x):
    assert isinstance(x, np.ndarray), "Input to ReLU derivative must be a numpy array"
    result = np.where(x > 0, 1, 0)
    assert np.all((result == 0) | (result == 1)), "ReLU derivative must be 0 or 1"
    return result

def softmax(x):
    assert isinstance(x, np.ndarray), "Input to softmax must be a numpy array"
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # pour stabilité numérique
    result = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    assert np.all((result >= 0) & (result <= 1)), "Softmax output must be in [0, 1]"
    assert np.allclose(np.sum(result, axis=1), 1), "Softmax output must sum to 1 per sample"
    return result

In [28]:
def compute_loss(Y_pred, Y_true, weights, lambda_):
    loss = -np.mean(Y_true * np.log(Y_pred) + (1 - Y_true) * np.log(1 - Y_pred))
    reg_term = 0
    for w in weights:
        reg_term += np.sum(w ** 2)
    reg_term *= lambda_ / 2
    return loss + reg_term

In [48]:
class MultiClassNeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.1,l2_lambda=0.0):
        """
        Initialize the neural network with given layer sizes and learning rate.
        layer_sizes: List of integers [input_size, hidden1_size,
        ..., output_size]
        """
        assert isinstance(layer_sizes,list) and len(layer_sizes) >= 2,"layer_sizes must be a list with at least 2 elements"
        assert all(isinstance(size, int) and size > 0 for size in layer_sizes), "All layer sizes must be positive integers"
        assert isinstance(learning_rate, (int, float)) and learning_rate > 0, "Learning rate must be a positive number"
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.weights = []
        self.biases = []
         # Initialisation des poids et biais
        np.random.seed(42)
        for i in range(len(layer_sizes)- 1):
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01
            b = np.zeros((1, layer_sizes[i+1]))
            assert w.shape == (layer_sizes[i], layer_sizes[i+1]),f"Weight matrix {i+1} has incorrect shape"
            assert b.shape == (1, layer_sizes[i+1]), f"Bias vector {i+1} has incorrect shape"
            self.weights.append(w)
            self.biases.append(b)
    def forward(self, X):
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
    
        self.activations = [X]
        self.z_values = []
    
        for i in range(len(self.weights) - 1):
            z = np.dot(self.activations[-1] , self.weights[i]) + self.biases[i]
            assert z.shape == (X.shape[0], self.layer_sizes[i + 1]), f"Z^{i+1} has incorrect shape"
            self.z_values.append(z)
            self.activations.append(relu(z))
    
        z = np.dot(self.activations[-1] , self.weights[-1]) + self.biases[-1]
        assert z.shape == (X.shape[0], self.layer_sizes[-1]), "Output Z has incorrect shape"
        self.z_values.append(z)
        output = softmax(z)
        assert output.shape == (X.shape[0], self.layer_sizes[-1]), "Output A has incorrect shape"
        self.activations.append(output)
    
        return self.activations [-1]
    def compute_loss(self, y_true, y_pred):
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to loss must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"
    
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        l2_penalty = self.l2_lambda / (2 * y_true.shape[0]) * sum(np.sum(w ** 2) for w in self.weights)
        assert not np.isnan(loss), "Loss computation resulted in NaN"
        return loss+l2_penalty
    def compute_accuracy(self, y_true, y_pred):
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to accuracy must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"
    
        predictions = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        accuracy = np.mean(predictions == true_labels)
        assert 0 <= accuracy <= 1, "Accuracy must be between 0 and 1"
        return accuracy
    def backward(self, X, y, outputs):
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray) and isinstance(outputs, np.ndarray), "Inputs to backward must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape == outputs.shape, "y and outputs must have the same shape"
    
        m = X.shape[0]
        self.d_weights = [np.zeros_like(w) for w in self.weights]
        self.d_biases = [np.zeros_like(b) for b in self.biases]
    
        dZ = outputs - y # Gradient pour softmax + cross - entropy
        assert dZ. shape == outputs .shape , "dZ for output layer has incorrect shape "
        self.d_weights[-1] = (self.activations[-2].T @ dZ) / m
        self.d_biases[-1] = np.sum(dZ, axis=0, keepdims=True) / m
    
        for i in range(len(self.weights) - 2, -1, -1):
            dZ = np.dot(dZ , self.weights[i + 1].T) * relu_derivative(self.z_values[i])
            assert dZ. shape == (X. shape [0] , self . layer_sizes [i +1]), f"dZ ^{[i +1]} has incorrect shape "
            self.d_weights[i] = (np.dot(self.activations[i].T , dZ)) / m
            self.d_biases[i] = np.sum(dZ, axis=0, keepdims=True) / m
        # TODO : Ajouter une r g u l a r i s a t i o n L2 aux gradients des poids
        # dW ^{[ l]} += lambda * W ^{[ l]} / m, o lambda est le coefficient de r g u l a r i s a t i o n
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * self.d_weights[i]
            self.biases[i] -= self.learning_rate * self.d_biases[i]
    def train(self, X, y, X_val, y_val, epochs, batch_size):
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray), "X and y must be numpy arrays"
        assert isinstance(X_val, np.ndarray) and isinstance(y_val, np.ndarray), "X_val and y_val must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape[1] == self.layer_sizes[-1], f"Output dimension ({y.shape[1]}) must match output layer size ({self.layer_sizes[-1]})"
        assert X_val.shape[1] == self.layer_sizes[0], f"Validation input dimension ({X_val.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y_val.shape[1] == self.layer_sizes[-1], f"Validation output dimension ({y_val.shape[1]}) must match output layer size ({self.layer_sizes[-1]})"
        assert isinstance(epochs, int) and epochs > 0, "Epochs must be a positive integer"
        assert isinstance(batch_size, int) and batch_size > 0, "Batch size must be a positive integer"
    
        train_losses, val_losses = [],[]
        train_accuracies, val_accuracies = [],[]
    
        for epoch in range(epochs):
            indices = np.random.permutation(X.shape[0])
            X_shuffled = X[indices]
            y_shuffled = y[indices]
    
            epoch_loss = 0
            for i in range(0, X.shape[0], batch_size):
                X_batch = X_shuffled[i:i+batch_size]
                y_batch = y_shuffled[i:i+batch_size]
    
                outputs = self.forward(X_batch)
                epoch_loss += self.compute_loss(y_batch, outputs)
                self.backward(X_batch, y_batch, outputs)
   #============================================================================================== 
            # Calculer les pertes et accuracies
            train_loss = epoch_loss / (X.shape[0] // batch_size)
            train_pred = self.forward(X)
            train_acc = self.compute_accuracy(y, train_pred)
            val_pred = self.forward(X_val)
            val_loss = self.compute_loss(y_val, val_pred)
            val_acc = self.compute_accuracy(y_val, val_pred)
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)
    
            if epoch % 10 == 0 or epoch == epochs - 1:
                print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")
    
        return train_losses, val_losses, train_accuracies, val_accuracies
    def predict(self, X):
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
    
        outputs = self.forward(X)
        predictions = np.argmax(outputs, axis=1)
        assert predictions.shape == (X.shape[0],), "Predictions have incorrect shape"
        return predictions
 #D f i n i r le chemin vers le dossier d c o m p r e s s
data_dir = os.path .join ( os . getcwd () , 'C:\\Users\\Admin\\Downloads\\amhcd-data-64\\tifinagh-images\\')
print ( data_dir )
current_working_directory = os . getcwd ()
print ( current_working_directory )
# Charger le fichier CSV contenant les tiquettes
try:
    labels_df = pd.read_csv ( os.path.join ( data_dir , 'amhcd-data-64/labels-map .csv ') )
    assert 'image_path ' in labels_df . columns and 'label ' in labels_df.columns , "CSV must contain ’ image_path ’ and ’ label ’ columns "
except FileNotFoundError :
    print ("labels-map.csv not found . Please check the dataset structure .")
    # Alternative : construire un DataFrame partir des dossiers
    image_paths = []
    labels = []
    for label_dir in os.listdir ( data_dir ):
        label_path = os.path . join ( data_dir , label_dir )
        if os . path.isdir ( label_path ) :
            for img_name in os.listdir ( label_path ) :
                image_paths . append ( os.path.join ( label_path ,img_name ))
                labels.append ( label_dir )
    labels_df = pd.DataFrame ({ 'image_path ': image_paths , 'label ':labels })
# V r i f i e r le DataFrame
assert not labels_df.empty , "No data loaded . Check dataset files ."
print (f" Loaded {len ( labels_df )} samples with { labels_df [ 'label '].nunique ()} unique classes .")

# Encoder les tiquettes
label_encoder = LabelEncoder ()
labels_df ['label_encoded'] = label_encoder . fit_transform ( labels_df['label '])
num_classes = len ( label_encoder . classes_ )
# Fonction pour charger et p r t r a i t e r une image
def load_and_preprocess_image ( image_path , target_size =(32 , 32) ):
    """
    Load and preprocess an image : convert to grayscale , resize normalize
    """
    assert os.path.exists ( image_path ) , f" Image not found :{ image_path }"
    img = cv2 . imread ( image_path , cv2 . IMREAD_GRAYSCALE )
    assert img is not None , f" Failed to load image : { image_path }"
    img = cv2.resize ( img , target_size )
    img = img.astype ( np . float32 ) / 255.0 # Normalisation
    return img.flatten () # Aplatir pour le r s e a u de neurones
# Charger toutes les images
X = np.array ([ load_and_preprocess_image ( os.path.join ( data_dir ,path )) for path in labels_df ['image_path ']])
y = labels_df ['label_encoded'].values
# V r i f i e r les dimensions
assert X.shape [0] == y.shape [0] , " Mismatch between number of images and labels "
assert X.shape [1] == 32 * 32 , f" Expected flattened image size of {32*32} , got {X.shape [1]} "

# Diviser en ensembles d’ e n t r a n e m e n t , validation et test
X_temp , X_test , y_temp , y_test = train_test_split (X , y , test_size =0.2 , stratify =y , random_state =42)
X_train , X_val , y_train , y_val = train_test_split ( X_temp , y_temp ,test_size =0.25 , stratify = y_temp , random_state =42)

# Convertir explicitement en NumPy arrays
X_train = np.array ( X_train )
X_val = np.array ( X_val )
X_test = np.array ( X_test )
y_train = np.array ( y_train )
y_val = np.array ( y_val )
y_test = np.array ( y_test )
assert X_train .shape [0] + X_val.shape [0] + X_test.shape [0] == X.shape [0] , "Train -val - test split sizes must sum to total samples"
print (f" Train : { X_train.shape [0]} samples , Validation : { X_val.shape [0]} samples , Test : { X_test.shape [0]} samples ")
# Encoder les tiquettes en one - hot pour la classification multiclasse
one_hot_encoder = OneHotEncoder ( sparse_output = False )
y_train_one_hot = np.array ( one_hot_encoder . fit_transform ( y_train.reshape ( -1 , 1) ))
y_val_one_hot = np.array ( one_hot_encoder.transform ( y_val.reshape( -1 , 1) ))
y_test_one_hot = np.array ( one_hot_encoder.transform ( y_test.reshape( -1 , 1) ))

# Verifier que les tableaux one - hot sont des NumPy arrays
assert isinstance ( y_train_one_hot , np.ndarray ) , " y_train_one_hot must be a numpy array "
assert isinstance ( y_val_one_hot , np.ndarray ) , " y_val_one_hot must be a numpy array "
assert isinstance ( y_test_one_hot , np.ndarray ) , " y_test_one_hot must be a numpy array "

# Creer et en treaner le modele
layer_sizes = [ X_train.shape [1] , 64 , 32 , num_classes ] # 64 et 32 neurones c a c h s , 33 classes
nn = MultiClassNeuralNetwork ( layer_sizes , learning_rate =0.01)
train_losses , val_losses , train_accuracies , val_accuracies = nn.train (X_train , y_train_one_hot , X_val , y_val_one_hot , epochs =100 ,batch_size =32)


# Predictions et valuation
y_pred = nn.predict ( X_test )
print ("\ nRapport de classification ( Test set ) :")
print ( classification_report ( y_test , y_pred , target_names =label_encoder . classes_ ))
# Matrice de confusion
cm = confusion_matrix ( y_test , y_pred )
plt.figure ( figsize =(10 , 8) )
sns.heatmap (cm , annot = True , fmt ='d', cmap ='Blues')
plt.title ('Matrice de confusion ( Test set)')
plt.xlabel ('Predit')
plt.ylabel ('Reel')
plt.savefig ('confusion_matrix.png')
plt.close ()

# Courbes de perte et d’accuracy
fig , ( ax1 , ax2 ) = plt.subplots (1 , 2, figsize =(12, 5) )

# Courbe de perte
ax1.plot ( train_losses , label ='Train Loss')
ax1.plot ( val_losses , label ='Validation Loss')
ax1.set_title ('Courbe de perte ')
ax1.set_xlabel ('poque')
ax1.set_ylabel ('Perte')
ax1.legend ()

# Courbe d’accuracy
ax2.plot ( train_accuracies , label ='Train Accuracy')
ax2.plot ( val_accuracies , label ='Validation Accuracy')
ax2.set_title ('Courbe de precision')
ax2.set_xlabel ('poque')
ax2.set_ylabel ('Prcision')
ax2.legend ()
plt.tight_layout ()
fig.savefig ('loss_accuracy_plot.png')
plt.close ()



<>:218: SyntaxWarning: invalid escape sequence '\ '
<>:218: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Admin\AppData\Local\Temp\ipykernel_14620\725336431.py:218: SyntaxWarning: invalid escape sequence '\ '
  print ("\ nRapport de classification ( Test set ) :")


C:\Users\Admin\Downloads\amhcd-data-64\tifinagh-images\
C:\Users\Admin\Downloads
labels-map.csv not found . Please check the dataset structure .
 Loaded 28182 samples with 33 unique classes .
 Train : 16908 samples , Validation : 5637 samples , Test : 5637 samples 
Epoch 0: Train Loss = 3.5033, Val Loss = 3.4965, Train Acc = 0.0396, Val Acc = 0.0373
Epoch 10: Train Loss = 3.5027, Val Loss = 3.4959, Train Acc = 0.0533, Val Acc = 0.0495
Epoch 20: Train Loss = 3.0726, Val Loss = 3.0363, Train Acc = 0.0966, Val Acc = 0.0953
Epoch 30: Train Loss = 2.3341, Val Loss = 2.3010, Train Acc = 0.3049, Val Acc = 0.3003
Epoch 40: Train Loss = 1.5625, Val Loss = 1.6036, Train Acc = 0.5117, Val Acc = 0.4882
Epoch 50: Train Loss = 1.0960, Val Loss = 1.1550, Train Acc = 0.6635, Val Acc = 0.6422
Epoch 60: Train Loss = 0.7771, Val Loss = 0.8465, Train Acc = 0.7711, Val Acc = 0.7367
Epoch 70: Train Loss = 0.5665, Val Loss = 0.6783, Train Acc = 0.8214, Val Acc = 0.7798
Epoch 80: Train Loss = 0.4245, Val Loss

In [46]:
layer_sizes = [X_train.shape [1] , 64 , 32 , num_classes ] # 64 et 32 neurones c a c h s , 33 classes
nn = MultiClassNeuralNetwork ( layer_sizes , learning_rate =0.1)
train_losses , val_losses , train_accuracies , val_accuracies = nn .train (X_train , y_train_one_hot , X_val , y_val_one_hot , epochs =100 ,batch_size =32 )

Epoch 0: Train Loss = 3.5041, Val Loss = 3.4962, Train Acc = 0.0303, Val Acc = 0.0303
Epoch 10: Train Loss = 0.4027, Val Loss = 0.5970, Train Acc = 0.8530, Val Acc = 0.8088
Epoch 20: Train Loss = 0.1244, Val Loss = 0.3238, Train Acc = 0.9593, Val Acc = 0.8975
Epoch 30: Train Loss = 0.0694, Val Loss = 0.3308, Train Acc = 0.9791, Val Acc = 0.9065
Epoch 40: Train Loss = 0.0315, Val Loss = 0.3529, Train Acc = 0.9830, Val Acc = 0.9124
Epoch 50: Train Loss = 0.0044, Val Loss = 0.2807, Train Acc = 0.9998, Val Acc = 0.9297
Epoch 60: Train Loss = 0.0017, Val Loss = 0.2819, Train Acc = 0.9999, Val Acc = 0.9310
Epoch 70: Train Loss = 0.0013, Val Loss = 0.2917, Train Acc = 0.9999, Val Acc = 0.9317
Epoch 80: Train Loss = 0.0008, Val Loss = 0.2978, Train Acc = 1.0000, Val Acc = 0.9331
Epoch 90: Train Loss = 0.0007, Val Loss = 0.3034, Train Acc = 1.0000, Val Acc = 0.9337
Epoch 99: Train Loss = 0.0010, Val Loss = 0.3078, Train Acc = 1.0000, Val Acc = 0.9337
